***
# Parse HTML to playlists.csv, users.csv, tracks.csv

## Needed Libraries

In [ ]:
import requests
import requests_cache
import random
from bs4 import BeautifulSoup
from time import sleep
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
# YOUR CODE HERE (OPTION) 
import datetime
# Nếu cần các thư viện khác thì bạn có thể import ở đây
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [ ]:
sleep_time = 1 # Đơn vị: giây

## Collect Playlist

*__collect_playlist__* func help us to collect playplists individually from existed url playlists to playlist.csv with a format of "*playlist's name + list of track's name*"  

Playlist's name      : get the name of the main playlist  
List of track's name : get list of tracks name according to current playlist.

Wrriten by: *__Nguyễn Trung Hiếu__*

In [ ]:
def collect_playlist(playlist_url, playlist_file):
    driver = webdriver.Chrome(executable_path=r'C:\Users\ASUS\Source_ipynb\chromedriver_win32\chromedriver.exe')  
    
    with open(playlist_file,'w', encoding='utf-8') as fo_playlist:
        with open(playlist_url,'r', encoding='utf-8') as fin:
            
            # column for each csv file
            fo_playlist.write('playlists\ttracks\n')
    
            # run per url to get playlist title and trackID
            for url_text in fin.readlines():
                time.sleep(sleep_time)
                
                driver.get(url_text.strip())
                html_text = driver.page_source
                
                soup = BeautifulSoup(html_text,'lxml')
                
                #get playlist title
                title = soup.find('h1',class_='soundTitle__title').text.strip()
                fo_playlist.write(title + "\t")
                

                #get trackIDs
                trackItem_title = soup.find_all('a',class_='trackItem__trackTitle')
                listTrackId = ''
               
                
                # get tracklist
                for index in range(0,len(trackItem_title)):
                        listTrackId += trackItem_title[index].text + ","     
   
                fo_playlist.write(listTrackId.rstrip(',') +"\n")
                sleep(random.randint(2,5))
                            
collect_playlist('playlist_urls_file.txt','playlists.csv')

## Collect Track

*__collect_track__* func help us to collect playplists individually from existed url track to tracks.csv with a format of "*Tracks + Play-repeated + heart + comment*"

Tracks     : track's name  
Play-reated: how many times has this track play repetitvely  
Heart      : number of heart giving by user  
Comment    : how many people comment on this track

Wrriten by: *__Nguyễn Trung Hiếu__*

In [ ]:
def collect_track(playlist_url, playlist_file):
    driver = webdriver.Chrome(executable_path=r'C:\Users\ASUS\Source_ipynb\chromedriver_win32\chromedriver.exe')  
    
    with open(playlist_file,'w', encoding='utf-8') as fo_playlist:
        with open(playlist_url,'r', encoding='utf-8') as fin:
            
            # column for each csv file
            fo_playlist.write('tracks\tplay-repeated\theart\tcomment\n')
    
            # run per url to get playlist title and trackID
            for url_text in fin.readlines():
                time.sleep(sleep_time)
                driver.get(url_text.strip())
                sleep(random.randint(3,6))
                html_text = driver.page_source
                
                soup = BeautifulSoup(html_text,'lxml')
                
                blockedTrack = soup.find('h1',class_='blockedTrackMessage')
                    
                if blockedTrack != None:
                    continue
            
                
                #get playlist title
                title = soup.find('h1',{'class','soundTitle__title'})
                
                if title == None:
                    continue
                
                fo_playlist.write(title.text.strip() + "\t")
                
                #get Play
                Time_Play = soup.find_all('span',class_='sc-ministats')
                fo_playlist.write(Time_Play[0].text.split()[0].strip() + "\t")
                
                #Get heart 
                heart = soup.find_all('li',class_='sc-ministats-item')
                fo_playlist.write(heart[1]['title'].split()[0].strip() + "\t")
                
                #Get comment
                comment = soup.find('span', class_='commentsList__actualTitle')
                fo_playlist.write(comment.text.split()[0].strip() + "\n")
                sleep(random.randint(7,10))
                            
collect_track('track_urls_file.txt','tracks.csv')

## Collect User

*__collect_user__* func help us to collect playplists individually from existed url user to users.csv with a format of "*user-name + Verified + Follower + Following + Track*"

User-name: user name  
Verified : check whether that user is verified or not  
Follower : number of followers
Following: number of other user-name did this user name follow
Track    : number of tracks has this user uploaded so far

Wrriten by: __*Trần Xuân Phước*__

In [ ]:
def collect_user(playlist_url, playlist_file):
    driver = webdriver.Chrome(executable_path=r'C:\Users\ASUS\Source_ipynb\chromedriver_win32\chromedriver.exe')  
    
    with open(playlist_file,'w', encoding='utf-8') as fo_playlist:
        with open(playlist_url,'r', encoding='utf-8') as fin:
            
            # column for each csv file
            fo_playlist.write('user-name\tVerified\tFollower\tFollowing\tTrack\n')
    
            # run per url to get playlist title and trackID
            for url_text in fin.readlines():
                time.sleep(sleep_time)
                
                driver.get(url_text.strip())
                sleep(random.randint(3,6))
                html_text = driver.page_source
                soup = BeautifulSoup(html_text,'lxml')
                
                #get user name 
                user_name = soup.find('h2',{'class','profileHeaderInfo__userName'})   
                a = ""
                is_verified = 0
                for text in user_name.text.split():
                    if "Verified" != text:
                        a += text
                    else:
                        is_verified = 1
                        fo_playlist.write(a + "\t1\t")
                if is_verified == 0:
                    fo_playlist.write(user_name.text.strip() + "\t0\t")
                
                #get Follower
                info = soup.find_all('div',class_='infoStats__value')
                fo_playlist.write(info[0].text.strip() + "\t")
                
                #Get Following 
                fo_playlist.write(info[1].text.strip() + "\t")
                
                #Get track
                fo_playlist.write(info[2].text.strip() + "\n")
                sleep(random.randint(7,10))
                            
collect_user('user_urls_file.txt','users.csv')